In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from sklearn.preprocessing import LabelEncoder,StandardScaler

#model selection from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
#ensemble
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier
#metrics
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
#pca
from sklearn.decomposition import PCA
#data_balance
from sklearn.utils import resample
#warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
pd.set_option('display.max_columns',100)

In [48]:
import os
os.chdir(r'C:\Users\Prabhakaran\Documents\GitHub\CAPSTONE\Data')

In [49]:
df=pd.read_csv('./train.csv')
df.head()

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,DisbursalDate,State_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,loan_default
0,420825,50578,58400,89.55,67,22807,45,1441,01-01-84,Salaried,03-08-18,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
1,537409,47145,65550,73.23,67,22807,45,1502,31-07-85,Self employed,26-09-18,6,1998,1,1,0,0,0,0,598,I-Medium Risk,1,1,1,27600,50200,50200,0,0,0,0,0,0,1991,0,0,1,1yrs 11mon,1yrs 11mon,0,1
2,417566,53278,61360,89.63,67,22807,45,1497,24-08-85,Self employed,01-08-18,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
3,624493,57513,66113,88.48,67,22807,45,1501,30-12-93,Self employed,26-10-18,6,1998,1,1,0,0,0,0,305,L-Very High Risk,3,0,0,0,0,0,0,0,0,0,0,0,31,0,0,0,0yrs 8mon,1yrs 3mon,1,1
4,539055,52378,60300,88.39,67,22807,45,1495,09-12-77,Self employed,26-09-18,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,1,1


In [50]:
df.shape

(233154, 41)

### Attribute description

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
UniqueID,233154.0,535917.573376,6.831569e+04,417428.00,476786.25,535978.5,595039.75,6.710840e+05
disbursed_amount,233154.0,54356.993528,1.297131e+04,13320.00,47145.00,53803.0,60413.00,9.905720e+05
asset_cost,233154.0,75865.068144,1.894478e+04,37000.00,65717.00,70946.0,79201.75,1.628992e+06
ltv,233154.0,74.746530,1.145664e+01,10.03,68.88,76.8,83.67,9.500000e+01
branch_id,233154.0,72.936094,6.983499e+01,1.00,14.00,61.0,130.00,2.610000e+02
supplier_id,233154.0,19638.635035,3.491950e+03,10524.00,16535.00,20333.0,23000.00,2.480300e+04
manufacturer_id,233154.0,69.028054,2.214130e+01,45.00,48.00,86.0,86.00,1.560000e+02
Current_pincode_ID,233154.0,3396.880247,2.238148e+03,1.00,1511.00,2970.0,5677.00,7.345000e+03
State_ID,233154.0,7.262243,4.482230e+00,1.00,4.00,6.0,10.00,2.200000e+01
Employee_code_ID,233154.0,1549.477148,9.752613e+02,1.00,713.00,1451.0,2362.00,3.795000e+03


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233154 entries, 0 to 233153
Data columns (total 41 columns):
UniqueID                               233154 non-null int64
disbursed_amount                       233154 non-null int64
asset_cost                             233154 non-null int64
ltv                                    233154 non-null float64
branch_id                              233154 non-null int64
supplier_id                            233154 non-null int64
manufacturer_id                        233154 non-null int64
Current_pincode_ID                     233154 non-null int64
Date.of.Birth                          233154 non-null object
Employment.Type                        225493 non-null object
DisbursalDate                          233154 non-null object
State_ID                               233154 non-null int64
Employee_code_ID                       233154 non-null int64
MobileNo_Avl_Flag                      233154 non-null int64
Aadhar_flag                   

## Data Wrangling

In [8]:
temp_col = ['PERFORM_CNS.SCORE.DESCRIPTION']
df[temp_col] = df[temp_col].replace('No Bureau History Available', 0)
df[temp_col] = df[temp_col].replace('Not Scored: Sufficient History Not Available',0)
df[temp_col] = df[temp_col].replace('Not Scored: Not Enough Info available on the customer',0)
df[temp_col] = df[temp_col].replace('Not Scored: No Activity seen on the customer (Inactive)',0) 
df[temp_col] = df[temp_col].replace('Not Scored: No Updates available in last 36 months',0) 
df[temp_col] = df[temp_col].replace('Not Scored: Only a Guarantor', 0)
df[temp_col] = df[temp_col].replace('Not Scored: More than 50 active Accounts found',0)
df[temp_col] = df[temp_col].replace('M-Very High Risk', 1)
df[temp_col] = df[temp_col].replace('L-Very High Risk', 1)
df[temp_col] = df[temp_col].replace('K-High Risk', 2)
df[temp_col] = df[temp_col].replace('J-High Risk', 2)
df[temp_col] = df[temp_col].replace('I-Medium Risk', 3)
df[temp_col] = df[temp_col].replace('H-Medium Risk', 3)
df[temp_col] = df[temp_col].replace('G-Low Risk', 4)
df[temp_col] = df[temp_col].replace('F-Low Risk', 4)
df[temp_col] = df[temp_col].replace('E-Low Risk', 4)
df[temp_col] = df[temp_col].replace('D-Very Low Risk', 5)
df[temp_col] = df[temp_col].replace('C-Very Low Risk', 5)
df[temp_col] = df[temp_col].replace('B-Very Low Risk', 5)
df[temp_col] = df[temp_col].replace('A-Very Low Risk', 5)

In [9]:
df['Employment.Type'].fillna('Others',inplace=True)
df['Employment.Type'].value_counts()
df['Employment.Type']=df['Employment.Type'].replace({'Self employed':0, 'Salaried':1 ,'Others':2})

In [10]:
df.rename(columns={'Employment.Type':'Employment_Type'},inplace=True)

In [11]:
def duration(dur):
    yrs = int(dur.split(' ')[0].replace('yrs',''))
    mon = int(dur.split(' ')[1].replace('mon',''))
    return yrs*12+mon

In [12]:
df['Avg_Loan_tenure'] = df['AVERAGE.ACCT.AGE'].apply(duration)
df['Time_since_1st_loan'] = df['CREDIT.HISTORY.LENGTH'].apply(duration)

In [13]:
df.drop(['AVERAGE.ACCT.AGE','CREDIT.HISTORY.LENGTH'],axis=1,inplace=True)

In [14]:
def age(dur):
    yr = int(dur.split('-')[2])
    if yr >=0 and yr<=19:
        return yr+2000
    else:
         return yr+1900

df['Date.of.Birth'] = df['Date.of.Birth'].apply(age)
df['DisbursalDate'] = df['DisbursalDate'].apply(age)

In [15]:
df['Age_at_disbursal']=df['DisbursalDate']-df['Date.of.Birth']

In [16]:
df.head()

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment_Type,DisbursalDate,State_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,NO.OF_INQUIRIES,loan_default,Avg_Loan_tenure,Time_since_1st_loan,Age_at_disbursal
0,420825,50578,58400,89.55,67,22807,45,1441,1984,1,2018,6,1998,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,34
1,537409,47145,65550,73.23,67,22807,45,1502,1985,0,2018,6,1998,1,1,0,0,0,0,598,3,1,1,1,27600,50200,50200,0,0,0,0,0,0,1991,0,0,1,0,1,23,23,33
2,417566,53278,61360,89.63,67,22807,45,1497,1985,0,2018,6,1998,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,33
3,624493,57513,66113,88.48,67,22807,45,1501,1993,0,2018,6,1998,1,1,0,0,0,0,305,1,3,0,0,0,0,0,0,0,0,0,0,0,31,0,0,0,1,1,8,15,25
4,539055,52378,60300,88.39,67,22807,45,1495,1977,0,2018,6,1998,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,41


## Base Model

In [17]:
y=df.loan_default
X=df.drop("loan_default",axis=1)
from sklearn.model_selection import train_test_split,KFold,cross_val_score
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [18]:
lr=LogisticRegression()
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)

In [19]:
# for Test set
model_score = lr.score(X_test,y_test)
print(model_score)

0.7827069066578981


In [20]:
# for train set
model_score = lr.score(X_train,y_train)
print(model_score)

0.7829382318160373


In [21]:
print("precision :",precision_score(y_test,y_pred),"\n")
print("f1 score:",f1_score(y_test,y_pred),"\n")

precision : 0.0 

f1 score: 0.0 



In [22]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.78      1.00      0.88     54756
           1       0.00      0.00      0.00     15191

    accuracy                           0.78     69947
   macro avg       0.39      0.50      0.44     69947
weighted avg       0.61      0.78      0.69     69947



In [23]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[54748,     8],
       [15191,     0]], dtype=int64)

## Feature Engineering

In [24]:
numerical = ['disbursed_amount','asset_cost','ltv','PRI.CURRENT.BALANCE','PRI.SANCTIONED.AMOUNT',
             'PRI.DISBURSED.AMOUNT','SEC.CURRENT.BALANCE','SEC.SANCTIONED.AMOUNT','SEC.DISBURSED.AMOUNT',
             'PRIMARY.INSTAL.AMT','SEC.INSTAL.AMT']

In [25]:
lower={}
upper={}
for i in numerical:
        q1=df[i].quantile(0.25)
        q3=df[i].quantile(0.75)
        iqr=q3-q1
        lower[i]=q1 - iqr * 1.5
        upper[i]=q3 + iqr * 1.5

In [26]:
df['flag']=0
for row in range(len(df)):
    count=0
    for col in numerical:
        if (df.loc[row,col] < lower[col]) or (df.loc[row,col] > upper[col]) :
            count+=1
        if count>=4:
            df.loc[row,'flag'] = 1

In [27]:
df['flag'].value_counts()

0    207341
1     25813
Name: flag, dtype: int64

In [28]:
df[(df['flag']==1) & (df['loan_default']==1)].shape

(4470, 43)

In [29]:
df.drop(df.loc[df['flag']==1].index,axis=0,inplace=True)

In [30]:
df.shape

(207341, 43)

In [31]:
y=df.loan_default
X=df.drop("loan_default",axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [32]:
lr=LogisticRegression()
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)

In [33]:
# for Test set
model_score = lr.score(X_test,y_test)
print(model_score)

0.7774544636110798


In [34]:
# for train set
model_score = lr.score(X_train,y_train)
print(model_score)

0.7774187325166393


In [35]:
print("precision :",precision_score(y_test,y_pred),"\n")
print("f1 score:",f1_score(y_test,y_pred),"\n")

precision : 0.0 

f1 score: 0.0 



In [36]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.78      1.00      0.87     48362
           1       0.00      0.00      0.00     13841

    accuracy                           0.78     62203
   macro avg       0.39      0.50      0.44     62203
weighted avg       0.60      0.78      0.68     62203



In [37]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[48360,     2],
       [13841,     0]], dtype=int64)

In [38]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=27, ratio=1.0)
X_train, y_train = sm.fit_sample(X_train, y_train)


### Scaling

In [39]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [40]:
lr=LogisticRegression()
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)

In [41]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.84      0.55      0.66     48362
           1       0.29      0.63      0.39     13841

    accuracy                           0.57     62203
   macro avg       0.56      0.59      0.53     62203
weighted avg       0.72      0.57      0.60     62203



In [42]:
confusion_matrix(y_test,y_pred)

array([[26652, 21710],
       [ 5153,  8688]], dtype=int64)

In [43]:
l=[]

In [44]:
from scipy.stats import ttest_ind

for i in numerical:
    df1=df.groupby('loan_default').get_group(0)
    df2=df.groupby('loan_default').get_group(1)
    t,pvalue=ttest_ind(df1[i],df2[i])
    print(i,pvalue)
    if pvalue>0.05:
        l.append(i)

disbursed_amount 0.0
asset_cost 6.962703223381257e-11
ltv 0.0
PRI.CURRENT.BALANCE 1.1095448238237146e-18
PRI.SANCTIONED.AMOUNT 0.6763007017238067
PRI.DISBURSED.AMOUNT 0.6951943868019048
SEC.CURRENT.BALANCE 0.13742752525269072
SEC.SANCTIONED.AMOUNT 0.09095690363540665
SEC.DISBURSED.AMOUNT 0.09359353116064206
PRIMARY.INSTAL.AMT 6.796279945152978e-06
SEC.INSTAL.AMT 0.7811666880440156


In [45]:
from scipy.stats import chi2_contingency

categorical = ['branch_id','supplier_id','manufacturer_id','Current_pincode_ID','Employment_Type',
              'State_ID','Employee_code_ID','MobileNo_Avl_Flag','Aadhar_flag','PAN_flag','VoterID_flag',
               'Driving_flag','Passport_flag','PERFORM_CNS.SCORE.DESCRIPTION','PRI.NO.OF.ACCTS','PRI.ACTIVE.ACCTS',
               'PRI.OVERDUE.ACCTS','SEC.NO.OF.ACCTS','SEC.ACTIVE.ACCTS','SEC.OVERDUE.ACCTS',
              'NEW.ACCTS.IN.LAST.SIX.MONTHS','DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS','NO.OF_INQUIRIES',
              'Avg_Loan_tenure','Time_since_1st_loan','Age_at_disbursal']
k=[]
for i in categorical:
    pvalue  = chi2_contingency(pd.crosstab(df['loan_default'],df[i]))[1]
    print(i,pvalue)
    if pvalue>0.05:
        l.append(i)

branch_id 0.0
supplier_id 0.0
manufacturer_id 2.7208694829659704e-83
Current_pincode_ID 8.804067591107e-312
Employment_Type 1.8885924652151902e-32
State_ID 1.1975626111688994e-308
Employee_code_ID 0.0
MobileNo_Avl_Flag 1.0
Aadhar_flag 1.5435570038773106e-74
PAN_flag 0.6172360078931455
VoterID_flag 2.4909324772822585e-81
Driving_flag 0.021468967723283588
Passport_flag 0.0034788056738400417
PERFORM_CNS.SCORE.DESCRIPTION 0.0
PRI.NO.OF.ACCTS 4.643717997388007e-23
PRI.ACTIVE.ACCTS 2.137122285785575e-26
PRI.OVERDUE.ACCTS 2.166987264203763e-120
SEC.NO.OF.ACCTS 0.2479954475098534
SEC.ACTIVE.ACCTS 0.24634739447239287
SEC.OVERDUE.ACCTS 0.5380106215225176
NEW.ACCTS.IN.LAST.SIX.MONTHS 1.8155413025523473e-22
DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS 9.711114272197416e-79
NO.OF_INQUIRIES 5.265732421421659e-88
Avg_Loan_tenure 5.0371980837129904e-11
Time_since_1st_loan 1.1455873798986276e-15
Age_at_disbursal 1.402175391865282e-33


In [46]:
l

['PRI.SANCTIONED.AMOUNT',
 'PRI.DISBURSED.AMOUNT',
 'SEC.CURRENT.BALANCE',
 'SEC.SANCTIONED.AMOUNT',
 'SEC.DISBURSED.AMOUNT',
 'SEC.INSTAL.AMT',
 'MobileNo_Avl_Flag',
 'PAN_flag',
 'SEC.NO.OF.ACCTS',
 'SEC.ACTIVE.ACCTS',
 'SEC.OVERDUE.ACCTS']

In [47]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif=[variance_inflation_factor(X.values,i) for i in range(X.shape[1])]
pd.DataFrame({'vif': vif}, index=X.columns)

,vif
UniqueID,1.068907
disbursed_amount,32.031148
asset_cost,29.367609
ltv,16.266774
branch_id,1.095792
supplier_id,1.128341
manufacturer_id,1.055573
Current_pincode_ID,1.350729
Date.of.Birth,inf
Employment_Type,1.099511
